## New York Health Care Claims Reports 2022

Here we analyze the New York State ealth care claims reports corresponding to plan years 2022.

At the time of writing, the following links host and provide information about these reports:
- https://www.dfs.ny.gov/reports_and_publications/health_care_claim_reports
- https://www.dfs.ny.gov/apps_and_licensing/health_care_claim_reports

### Setup

In [202]:
import os
import sys

import numpy as np
import pandas as pd
import scrapy
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from datetime import datetime


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
sys.path.append("../..")

from src.utils.data.clean_and_parse import cast_to_int, cast_to_float
from src.utils.download import download_file_from_url
from src.utils.plot import plot_bar, plot_pie, heatmap_from_df_cols

INPUT_DATA_DIR = "./input_data"
OUTPUT_IMAGE_DIR = "./images/ny_claim_reports"
OUTPUT_DATA_DIR = "./output_data/ny_claim_reports"  # Set to None if you don't want to save non-required data for external use

os.makedirs(INPUT_DATA_DIR, exist_ok=True)
if OUTPUT_IMAGE_DIR:
    os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)
if OUTPUT_DATA_DIR:
    os.makedirs(OUTPUT_DATA_DIR, exist_ok=True)

In [203]:
# DL utilities

import requests
from bs4 import BeautifulSoup


def scrape_table_from_url(url):
    # Send a GET request to the URL
    response = requests.get(url)

    if response.status_code != 200:
        # If the request was not successful, raise an exception
        raise Exception(f"Error accessing URL: {response.status_code}")

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table element(s) in the HTML
    table = soup.find("table")

    if not table:
        # If no table was found, raise an exception or return None
        raise Exception("No table found on the page")

    # Process the table data and extract the rows and columns
    rows = table.find_all("tr")
    table_data = {}

    for row in rows:
        columns = row.find_all("td")
        if len(columns) == 0:
            continue
        row_name = columns[0].text.strip()
        row_data = columns[-1].find("a").get("href")
        table_data[row_name] = "https://www.dfs.ny.gov" + row_data

    return table_data


import shutil


def download_file(url, fname):
    local_filename = url.split("/")[-1]
    with requests.get(url, stream=True) as r:
        with open(fname, "wb") as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

### Download Raw Data

In [204]:
claims_report_links = scrape_table_from_url(
    "https://www.dfs.ny.gov/reports_and_publications/health_care_claim_reports"
)

# URL obtained by clicking "Export" here: https://www.dfs.ny.gov/public-appeal/search
key = list(claims_report_links.keys())[-1]
url = claims_report_links[key]
appeal_data_path = os.path.join(INPUT_DATA_DIR, f"{key}.xlsx")

download_file(url, fname=appeal_data_path)

xlsx = pd.ExcelFile(appeal_data_path)
# xlsx.sheet_names

In [205]:
df = pd.read_excel(xlsx, sheet_name="Commercial")

### NYS Annual Reports Template parsing


In [207]:
# Note: legend says all instances of 0 mean no data! Not sure if this means unknown, or no data as in no (zero) claims.

In [264]:
# Constuct global map from insurers to NAIC and relevant commercial claims DF
commercial = {}

# Loop
name = df.columns[1]
commercial[name] = {}

naic_num = df.iloc[0][1]
commercial[name]["naic_company_code"] = naic_num

df = pd.read_excel(xlsx, sheet_name="Commercial")
commercial[name]["claims_df"] = df

In [265]:
def extract_claims_df(df: pd.DataFrame) -> pd.DataFrame:
    # Extract claims df of interest

    # Isolate rows and cols of interest, transpose
    claim_data = df.iloc[6:26]
    claim_data = claim_data.iloc[:, 2:11].T.reset_index(drop=True)

    # Set col names
    claim_data.columns = claim_data.iloc[0]
    claim_data = claim_data.tail(-1)

    # Reset cols
    mapper = {
        "Major Category of Health Care Provider": "provider_category",
        "# of Claims Received during the Current Reporting Period": "new_claims_recvd",
        "$ Value of Claims Received during the Current Reporting Period (Billed Charges)": "new_claims_recvd_billed",
        "$ Value of Claims Received during the Current Reporting Period (Allowed Amount)": "new_claims_recvd_allowed",
        "# of Claims Pended at the end of the Prior Reporting Period": "pending_claims_recvd",
        "$ Value of Claims Pended at the end of the Prior Reporting Period (Billed Charges)": "pending_claims_recvd_billed",
        "$ Value of Claims Pended at the end of the Prior Reporting Period (Allowed Amount)": "pending_claims_recvd_allowed",
        "# of Claims Rejected during the Current Reporting Period": "claims_rejected",
        "# of Claims Paid in Full during the Current Reporting Period": "claims_paid",
        "$ Value of Claims Paid in Full during the Current Reporting Period (Billed Charges)": "claims_paid_billed",
        "$ Value of Claims Paid in Full during the Current Reporting Period (Allowed Amount)": "claims_paid_allowed",
        "# of Claims Denied in Full during the Current Reporting Period": "claims_denied",
        "$ Value of Claims Denied in Full during the Current Reporting Period (Billed Charges)": "claims_denied_billed",
        "$ Value of Claims Denied in Full during the Current Reporting Period (Allowed Amount)": "claims_denied_allowed",
        "# of Claims Paid in Part and Denied in Part during the Current Reporting Period": "claims_partially_paid",
        "$ Value of Claims Partially Paid during the Current Reporting Period (Billed Charges)": "claims_partially_paid_billed",
        "$ Value of Claims Partially Paid during the Current Reporting Period (Allowed Amount)": "claims_partially_paid_allowed",
        "$ Value of Claims Partially Denied during the Current Reporting Period (Billed Charges)": "claims_partially_denied_billed",
        "$ Value of Claims Partially Denied during the Current Reporting Period (Allowed Amount)": "claims_partially_denied_allowed",
        "# of Claims Pended at the end of the Current Reporting Period": "claims_left_pending",
    }
    claim_data = claim_data.rename(columns=mapper)

    # Cast dtypes
    int_cols = [
        "new_claims_recvd",
        "pending_claims_recvd",
        "claims_rejected",
        "claims_paid",
        "claims_denied",
        "claims_partially_paid",
        "claims_left_pending",
    ]
    float_cols = [
        "new_claims_recvd_billed",
        "new_claims_recvd_allowed",
        "pending_claims_recvd_billed",
        "pending_claims_recvd_allowed",
        "claims_paid_billed",
        "claims_paid_allowed",
        "claims_denied_billed",
        "claims_denied_allowed",
        "claims_partially_paid_billed",
        "claims_partially_paid_allowed",
        "claims_partially_denied_billed",
        "claims_partially_denied_allowed",
    ]
    claim_data = cast_to_int(claim_data, cols=int_cols)
    claim_data = cast_to_float(claim_data, cols=float_cols)

    # Sanity check tests

    # Totals are in fact correct for each provider category
    for col in claim_data.columns[1:]:
        assert claim_data[col].values[:-1].sum() == claim_data[col].values[-1]

    # Claims in = Claims out + Claims processed (in each cat)
    pd.testing.assert_series_equal(
        claim_data["new_claims_recvd"] + claim_data["pending_claims_recvd"],
        claim_data["claims_rejected"]
        + claim_data["claims_paid"]
        + claim_data["claims_denied"]
        + claim_data["claims_partially_paid"]
        + claim_data["claims_left_pending"],
    )

    claim_data["claims_recvd"] = (
        claim_data["new_claims_recvd"] + claim_data["pending_claims_recvd"]
    )

    claim_data["claims_adjudicated"] = (
        claim_data["claims_recvd"] - claim_data["claims_left_pending"]
    )

    return claim_data

In [318]:
def extract_appeals_df(df: pd.DataFrame) -> pd.DataFrame:
    # Extract appeals df of interest

    # Isolate rows and cols of interest, transpose
    appeal_data = df.iloc[34:49]
    appeal_data = appeal_data.iloc[:, 2:11].T.reset_index(drop=True)

    # Set col names
    appeal_data.columns = appeal_data.iloc[0]
    appeal_data = appeal_data.tail(-1)

    # Reset cols
    mapper = {
        "Major Category of Health Care Provider": "provider_category",
        "# of First Level Appeals Received during the Current Reporting Period": "new_internal_appeals",
        "$ Value of Claims Initially Denied in Full or in Part that were submitted for First Level Appeal during the Current Reporting Period (Billed Charges)": "new_internal_appeals_billed",
        "$ Value of Claims Initially Denied in Full or in Part that were submitted for First Level Appeal during the Current Reporting Period (Allowed Charges)": "new_internal_appeals_allowed",
        "# of First Level Appeals Pending at the end of the Prior Reporting Period": "pending_internal_appeals",
        "$ Value of Claims Initially Denied in Full or in Part that were pending First Level Appeal at the end of the Prior Reporting Period (Billed Charges)": "pending_internal_appeals_billed",
        "$ Value of Claims Initially Denied in Full or in Part that were pending First Level Appeal at the end of the Prior Reporting Period (Allowed Amount)": "pending_internal_appeals_allowed",
        "# of First Level Appeals Overturned in Full during the Current Reporting Period": "internal_appeals_overturned",
        "# of First Level Appeals Overturned in Part during the Current Reporting Period": "internal_appeals_partially_overturned",
        "$ Value of Claims Paid in Full or in Part upon First Level Appeal during the Current Reporting Period (Billed Charges)": "internal_appeals_overturned_billed",
        "$ Value of Claims Paid in Full or in Part upon First Level Appeal during the Current Reporting Period (Allowed Amount)": "internal_appeals_overturned_allowed",
        "# of First Level Appeals Upheld during the Current Reporting Period": "internal_appeals_upheld",
        "$ Value of Claims Denied in Full or in Part upon First Level Appeal during the Current Reporting Period (Billed Charges)": "internal_appeals_upheld_billed",
        "$ Value of Claims Denied in Full or in Part upon First Level Appeal during the Current Reporting Period (Allowed Amount)": "internal_appeals_upheld_allowed",
        "# of First Level Appeals Pending at the end of the Current Reporting Period": "internal_appeals_left_pending",
    }
    appeal_data = appeal_data.rename(columns=mapper)

    # Cast dtypes
    int_cols = [
        "new_internal_appeals",
        "pending_internal_appeals",
        "internal_appeals_overturned",
        "internal_appeals_partially_overturned",
        "internal_appeals_upheld",
        "internal_appeals_left_pending",
    ]
    float_cols = [
        "new_internal_appeals_billed",
        "new_internal_appeals_allowed",
        "pending_internal_appeals_billed",
        "pending_internal_appeals_allowed",
        "internal_appeals_overturned_billed",
        "internal_appeals_overturned_allowed",
        "internal_appeals_upheld_billed",
        "internal_appeals_upheld_allowed",
    ]

    appeal_data = appeal_data.replace(pd.NA, 0)
    appeal_data = cast_to_int(appeal_data, cols=int_cols)
    appeal_data = cast_to_float(appeal_data, cols=float_cols)

    # Sanity check test

    # totals
    accounted_cols = [
        "pending_internal_appeals",
        "new_internal_appeals",
        "internal_appeals_overturned",
        "internal_appeals_partially_overturned",
        "internal_appeals_upheld",
        "internal_appeals_left_pending",
    ]
    for col in accounted_cols:
        assert appeal_data[col].values[:-1].sum() == appeal_data[col].values[-1]

    # Appeals in = appeals out + appeals processed (in each cat)
    pd.testing.assert_series_equal(
        appeal_data["new_internal_appeals"] + appeal_data["pending_internal_appeals"],
        appeal_data["internal_appeals_overturned"]
        + appeal_data["internal_appeals_partially_overturned"]
        + appeal_data["internal_appeals_upheld"]
        + appeal_data["internal_appeals_left_pending"],
    )

    appeal_data["internal_appeals_recvd"] = (
        appeal_data["new_internal_appeals"] + appeal_data["pending_internal_appeals"]
    )

    appeal_data["internal_appeals_adjudicated"] = (
        appeal_data["internal_appeals_recvd"]
        - appeal_data["internal_appeals_left_pending"]
    )

    return appeal_data

In [319]:
def extract_denial_rationales(df: pd.DataFrame) -> pd.DataFrame:
    # Extract denial rationales df of interest

    # Isolate rows and cols of interest, transpose
    df = df.iloc[58:].reset_index(drop=True)

    # Set col names
    df.columns = df.iloc[0]
    df = df.tail(-1).reset_index(drop=True)

    # Get NMN billed, allowed amts, total billed, allowed amts in each provider category
    nmn_billed = []
    nmn_allowed = []
    total_billed = []
    total_allowed = []
    for idx, row in df.iterrows():
        if idx % 3 == 1:
            nmn_billed.append(
                row[
                    "Not medically necessary (including experimental/investigational, clinical trial, rare disease)"
                ]
            )
            total_billed.append(row["Total"])
            df.drop(idx, inplace=True)
        if idx % 3 == 2:
            nmn_allowed.append(
                row[
                    "Not medically necessary (including experimental/investigational, clinical trial, rare disease)"
                ]
            )
            total_allowed.append(row["Total"])
            df.drop(idx, inplace=True)

    df["claims_denied_nmn_billed"] = nmn_billed
    df["claims_denied_nmn_allowed"] = nmn_allowed
    df["claims_denied_billed2"] = total_billed
    df["claims_denied_allowed2"] = total_allowed
    df = df.reset_index(drop=True)

    df = df.drop(columns=["Line", "Claim Denial Reasons"])

    mapper = {
        "Provider Category": "provider_category",
        "Not medically necessary (including experimental/investigational, clinical trial, rare disease)": "claims_denied_nmn_exp",
        "Non-formulary drug": "claims_denied_formulary",
        "Step therapy denial": "claims_denied_step",
        "Not a covered benefit/Exceeds benefit limits (e.g., visit limits)": "claims_denied_not_covered",
        "Failure of provider to request preauthorization": "claims_denied_auth",
        "Provider not in-network (no OON coverage)": "claims_denied_oon",
        "Member's coverage not in effect at time of service": "claims_denied_effective",
        "Claim not timely": "claims_denied_timely",
        "Duplicate Claim": "claims_denied_duplicate",
        "Coding issue": "claims_denied_coding",
        "Other": "claims_denied_other",
        "Total": "claims_denied2",
        "claims_denied_nmn_billed": "claims_denied_nmn_billed",
        "claims_denied_nmn_allowed": "claims_denied_nmn_allowed",
        "claims_denied_billed2": "claims_denied_billed2",
        "claims_denied_allowed2": "claims_denied_allowed2",
    }
    df = df.rename(columns=mapper)

    # Cast dtypes
    int_cols = [
        "claims_denied_nmn_exp",
        "claims_denied_formulary",
        "claims_denied_step",
        "claims_denied_not_covered",
        "claims_denied_auth",
        "claims_denied_oon",
        "claims_denied_effective",
        "claims_denied_timely",
        "claims_denied_duplicate",
        "claims_denied_coding",
        "claims_denied_other",
        "claims_denied2",
    ]
    float_cols = [
        "claims_denied_nmn_billed",
        "claims_denied_nmn_allowed",
        "claims_denied_billed2",
        "claims_denied_allowed2",
    ]

    df = df.replace(pd.NA, 0)
    df = cast_to_int(df, cols=int_cols)
    df = cast_to_float(df, cols=float_cols)

    return df

In [325]:
claim_data = extract_claims_df(df)
appeal_data = extract_appeals_df(df)
rationale_df = extract_denial_rationales(df)

insurer_df = claim_data.merge(appeal_data, on=["provider_category"], how="outer").merge(
    rationale_df, on=["provider_category"], how="outer"
)
insurer_df[["claims_denied", "claims_denied2"]]

,claims_denied,claims_denied2
0,7964.0,9313.0
1,14838.0,16621.0
2,23890.0,NaN
3,45356.0,50338.0
4,52012.0,57558.0
5,62971.0,62971.0
6,0.0,NaN
7,207031.0,NaN
8,NaN,26325.0
9,NaN,0.0


In [ ]:
# TODO: incorporate pending s.t. claims received + pending_start = pending_end + (claims_paid + claims_decided otherwise)
# Same for appeals

# Ultimately just want to know: "of the claims decided in this period (some combo of cols), what was denial rate, etc."